In [1]:
import pandas as pd
import re
from unidecode import unidecode

In [2]:
benefit_map = {
    'Assistência médica':                    ['medica', 'saude', 'medico'],
    'Assistência odontológica':              ['odonto', 'dentista'],
    'Assistência psicológia':                ['psicolog', 'saude mental'],
    'Auxílio academia':                      ['academia', 'gympass', 'gym', 'total pass'],
    'Auxílio combustível':                   ['combustivel'],
    'Auxílio creche':                        ['creche'],
    'Auxílio desenvolvimento':               ['desenvolvimento'],
    'Auxílio estacionamento':                ['estacionamento'],
    'Auxílio farmácia':                      ['farmacia', 'medicamento'],
    'Auxílio fretado':                       ['fretado'],
    'Auxílio home office':                   ['auxilio home', 'custo home'],
    'Bicicletário':                          ['bicleta'],
    'Bolsa auxílio':                         ['bolsa auxilio'],
    'Café da manhã':                         ['cafe da manha'],
    'Cesta básica':                          ['cesta basica'],
    'Cesta de natal':                        ['natal'],
    'Clube de vantagens':                    ['vantagens'],
    'Consignado':                            ['consignado'],
    'Convênio com empresas parceiras':       ['convenio'],
    'Cooperativa de crédito':                ['cooperativa de credito'],
    'Day off aniversário':                   ['aniversario'],
    'Desconto em produtos':                  ['desconto em produtos'],
    'Ginástica laboral':                     ['ginastica laboral'],
    'Horário flexível':                      ['horario flexivel', 'flexibilidade'],
    'Massoterapia':                          ['massoterapia', 'massagem'],
    'Participação nos Lucros ou Resultados': ['lucros', 'plr', 'ppr'],
    'Plano de Aquisição de Ações':           ['aquisicao de acoes'],
    'Previdência privada':                   ['previdencia'],
    'Programa de remuneração variável':      ['remuneracao variavel', 'bonificac', 'premiac'],
    'Programa de treinamentos':              ['treinamento', 'capacitacao'],
    'Refeitório':                            ['refeitorio'],
    'Restaurante interno':                   ['restaurante'],
    'Sala de Jogos':                         ['jogos'],
    'Seguro de vida':                        ['vida'],
    'Vale-alimentação':                      ['alimentacao'],
    'Vale-cultura':                          ['vale cultura', 'vale-cultura'],
    'Vale-refeição':                         ['refeicao', 'ticket'],
    'Vale-transporte':                       ['transporte'],
}

In [3]:
def get_senioridade(df):
    mapa_senioridade = {
        'Júnior': ['Júnior', 'Junior', 'Jr', 'Estágio', 'Estagiário', 'Trainee', 'trainne', 'I'],
        'Pleno': ['Pleno', 'Plena', 'Pl', 'Jr/Pl', 'II'],
        'Sênior': ['Sênior', 'Senior', 'Sr', 'Pl/Sr', 'Tech Lead', 'Líder', 'Especialista', 'III', 'Coordenadora']
    }

    # Inicializa todas as linhas com 'Não Informado'
    df['senioridade'] = 'Não Informado'

    for index, row in df.iterrows():
        for senioridade, termos in mapa_senioridade.items():
            for termo in termos:
                if re.search(r'\b' + termo + r'\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                   re.search(r'\b' + termo + r'\b', row['descricao'], flags=re.IGNORECASE):
                    df.at[index, 'senioridade'] = senioridade
                    break
            if df.at[index, 'senioridade'] != 'Não Informado':
                break
    return df

In [4]:
def identificar_cargo(df):
    for index, row in df.iterrows():
        cargo = None

        if re.search(r'\bCientista\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
           re.search(r'\bCientista\b', row['descricao'], flags=re.IGNORECASE) or \
           re.search(r'\bScientist\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
           re.search(r'\bScientist\b', row['descricao'], flags=re.IGNORECASE):

            cargo = 'Cientista de Dados'

        elif re.search(r'\bAnalista\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                re.search(r'\bAnalista\b', row['descricao'], flags=re.IGNORECASE):

            cargo = 'Analista de Dados'

        elif re.search(r'\bEngenheir\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                re.search(r'\bEngenheir\b', row['descricao'], flags=re.IGNORECASE):

            cargo = 'Engenheiro de Dados'

        else:
            cargo = 'Não identificado'

        df.at[index, 'posicao'] = cargo

    return df

In [5]:
def identificar_regime(df):
    for index, row in df.iterrows():
        regime = None

        if re.search(r'\bPJ\b', row['descricao'], flags=re.IGNORECASE):
            regime = 'PJ'
        else:
            regime = 'CLT'

        df.at[index, 'regime'] = regime

    return df

In [6]:
def format_benefits_list(description: str) -> list[str] | None:
    """Extrai e padroniza os benefícios a partir da descrição da vaga.

    Args:
        description (str): Descrição da vaga.

    Returns:
        list[str] | None: Lista com os benefícios padronizados, ou None se não forem encontrados.

    Exemplos:
    ---
    >>> format_benefits_list('Oferecemos vale-alimentação e plano de saúde.')
    ['Vale-alimentação', 'Plano de saúde']
    """

    try:
        # Inicializa uma lista para armazenar os benefícios extraídos
        extracted_benefits = []

        # Converte toda a descrição para minúsculas para facilitar a correspondência
        description_lower = description.lower()

        # Percorre o mapa de benefícios para verificar a presença de cada benefício na descrição
        for benefit, synonyms in benefit_map.items():
            # Verifica se algum sinônimo do benefício está presente na descrição
            if any(synonym.lower() in description_lower for synonym in synonyms):
                extracted_benefits.append(benefit)

        # Retorna a lista de benefícios extraídos, removendo duplicatas e mantendo a ordem original
        if extracted_benefits:
            return list(dict.fromkeys(extracted_benefits))
        else:
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [7]:
def get_skills_list(description: str) -> list[str] | None:
    try:
        skills_str = description
        skills_str = re.sub(r'\s+', ' ', skills_str)
        skills_str = re.sub(r'(?<=[a-z]|\))([A-Z])', r';\1', skills_str)
        skills_list = re.findall(r'(?:(?<=[\:\;\.\•\*]).*?[A-Z].*?(?=[\.\;\•\*\)]))', skills_str, flags=re.I)
        skills_list = list(map(lambda skill: skill.strip(), skills_list))
        return skills_list
    except:
        return None

def format_skills_list(skills_list: list[str], skills_map: dict[str, list]) -> list[str] | None:
    try:
        skills_list_formatted = []
        skills_str = ', '.join(skills_list)
        skills_str = replace_specifics_substrings(skills_str)
        skills_str = unidecode(skills_str).lower()

        for skills_key, skills_matches in skills_map.items():
            match_look = any(True if skill_word in skills_str else False for skill_word in skills_matches)
            if match_look:
                skills_list_formatted.append(skills_key)
            else:
                continue

        if len(skills_list_formatted) == 0:
            return None

        return skills_list_formatted

    except:
        return None

def replace_specifics_substrings(benefits_str: str) -> str:

    va_vr_matches = re.findall(r'V[R|A]\b', benefits_str)

    va_vr_map = zip(['Vale refeição', 'Vale alimentação'], ['VR', 'VA'])

    for va_vr_full, va_vr_short in va_vr_map:

        if va_vr_short in va_vr_matches:
            benefits_str = benefits_str.replace(va_vr_short, va_vr_full)

    return benefits_str

def build_skills_map(data, macro:bool = False):
    dict_micro_tema = {}
    dict_macro_tema = {}

    for index, row in data[['micro','palavras_chave']].iterrows():
        dict_micro_tema[row['micro']] = row['palavras_chave'].split(', ')

    if macro:
        df_macro_temas = data[['macro','micro']].groupby('macro').agg(palavras_chave = ('micro',lambda x: ', '.join(x.str.lower()))).reset_index()
        for index, row in df_macro_temas.iterrows():
            dict_macro_tema[row['macro']] = row['palavras_chave'].split(', ')
        return dict_macro_tema, dict_micro_tema
    else:
        return dict_micro_tema


In [8]:
df = pd.read_csv('../data/data_raw/vagas_glassdoor_raw.csv')

df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills
0,Glassdoor,https://www.glassdoor.com.br/job-listing/anali...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-12,NaN,2024-03-13,Engenheiro de dados,NaN,Analista de Dados Pleno (Data Science).,Quod,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009183402795,Trabalhar na área de dados na QUOD é empolgant...,"['Statistics', 'Databases', 'SQL', 'Security',..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-08,NaN,2024-03-13,Engenheiro de dados,NaN,Cientista de Dados - Inteligência Artificial,"LBCA - Lee, Brock, Camargo Advogados",NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009177642279,"A LBCA é um escritório full service, formado p...","['TensorFlow', 'Statistics', 'Computer science..."
2,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-28,NaN,2024-03-13,Cientista de dados,NaN,CIENTISTA DE DADOS PLENO - MONITORAMENTO (vaga...,ClearSale,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009147683343,JOB DESCRIPTIONA Área de Modelagem da ClearSal...,"['Statistics', 'E-commerce', 'SQL', 'Statistic..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-05,NaN,2024-03-13,Cientista de dados,NaN,Cientista de Dados - Vaga Afirmativa para Mulh...,"Vagas, empregos, oportunidades",NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009159657667,Na posição de Cientista de Dados na Conta Azul...,"['Statistics', 'SQL', 'Math', 'Machine learnin..."
4,Glassdoor,https://www.glassdoor.com.br/job-listing/cas-c...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-02,NaN,2024-03-13,Engenheiro de dados,NaN,CAS | Cientista de Dados - Foco em Crédito,Sicredi,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009055258960,Integrar os times de dados da Diretoria Execut...,"['Statistics', 'Cloud infrastructure', 'Relati..."


In [9]:
df = df.drop_duplicates(subset=['codigo_vaga'])

df = get_senioridade(df)
df = identificar_cargo(df)
df = identificar_regime(df)
df['senioridade'].replace('NaN', 'Não informado')
df['modalidade'] = df['modalidade'].replace('trabalho remoto', 'Remoto')
df['modalidade'] = df['modalidade'].fillna('Presencial')
df['estado'] = df['estado'].replace('Distrito Federal', 'DF')
df['modalidade'] = df['modalidade'].replace('Trabalho remoto', 'Remoto')
df.loc[df['modalidade'] == 'Remoto', 'cidade'] = 'Não informado'
df.loc[df['modalidade'] == 'Remoto', 'estado'] = 'Todo o Brasil'

df['beneficios'] = df['descricao'].apply(format_benefits_list)

/tmp/ipykernel_2729098/1671119655.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'CLT' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'regime'] = regime


In [10]:
dict_skills = pd.read_excel('../data/dicionario-skills.xlsx', sheet_name='Habilidades')
skills_map_macro, skills_map_micro = build_skills_map(dict_skills, macro=True)

df['habilidades_micro'] = df['descricao'].apply(lambda descricao: format_skills_list(get_skills_list(descricao), skills_map_micro))
df['habilidades_macro'] = df['habilidades_micro'].apply(lambda habilidade: format_skills_list(habilidade, skills_map_macro))

In [12]:
df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,...,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills,beneficios,habilidades_micro,habilidades_macro
0,Glassdoor,https://www.glassdoor.com.br/job-listing/anali...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-12,NaN,2024-03-13,Analista de Dados,Pleno,Analista de Dados Pleno (Data Science).,Quod,...,Remoto,NaN,CLT,NaN,1009183402795,Trabalhar na área de dados na QUOD é empolgant...,"['Statistics', 'Databases', 'SQL', 'Security',...","[Assistência odontológica, Auxílio academia, A...","[Teradata, SAS, Banco de dados, Estatística, E...","[Analytics, Banco de Dados, ETL, Engenharia de..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-08,NaN,2024-03-13,Cientista de Dados,Não Informado,Cientista de Dados - Inteligência Artificial,"LBCA - Lee, Brock, Camargo Advogados",...,Remoto,NaN,CLT,NaN,1009177642279,"A LBCA é um escritório full service, formado p...","['TensorFlow', 'Statistics', 'Computer science...","[Auxílio creche, Auxílio desenvolvimento, Prog...","[EDA, SAS, Estatística, Machine Learning, Deep...","[Analytics, Deep Learning, Machine Learning, P..."
2,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-28,NaN,2024-03-13,Cientista de Dados,Pleno,CIENTISTA DE DADOS PLENO - MONITORAMENTO (vaga...,ClearSale,...,Remoto,NaN,CLT,NaN,1009147683343,JOB DESCRIPTIONA Área de Modelagem da ClearSal...,"['Statistics', 'E-commerce', 'SQL', 'Statistic...","[Assistência médica, Assistência odontológica,...","[EDA, SAS, Git, Estatística, Machine Learning,...","[Analytics, Engenharia de Software, Machine Le..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-05,NaN,2024-03-13,Cientista de Dados,Não Informado,Cientista de Dados - Vaga Afirmativa para Mulh...,"Vagas, empregos, oportunidades",...,Remoto,NaN,CLT,NaN,1009159657667,Na posição de Cientista de Dados na Conta Azul...,"['Statistics', 'SQL', 'Math', 'Machine learnin...","[Assistência médica, Assistência odontológica,...","[SAS, GCP, Estatística, Machine Learning, Paco...","[Analytics, Cloud, Machine Learning, Pacote Of..."
4,Glassdoor,https://www.glassdoor.com.br/job-listing/cas-c...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-02,NaN,2024-03-13,Cientista de Dados,Não Informado,CAS | Cientista de Dados - Foco em Crédito,Sicredi,...,Remoto,NaN,CLT,NaN,1009055258960,Integrar os times de dados da Diretoria Execut...,"['Statistics', 'Cloud infrastructure', 'Relati...","[Assistência odontológica, Auxílio creche, Aux...","[EDA, SAS, Banco de dados, Git, Estatística, M...","[Analytics, Banco de Dados, Engenharia de Soft..."


In [13]:
df['senioridade'].value_counts()

senioridade
Não Informado    3487
Júnior           2233
Sênior           2200
Pleno            1719
Name: count, dtype: int64

In [14]:
df.to_excel('../data/data_clean/vagas_glassdoor_clean.xlsx', index=False)